# **PREPROCESSING DATASET**

## **INSTALL AND IMPORT LIBRARY**

In [ ]:
!pip -q install datasets
!pip -q install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import train_test_split
from google.colab import files
from google.colab import drive
import pandas as pd
import re
import os
import numpy as np
import tensorflow as tf
import shutil
from sklearn.metrics import classification_report, accuracy_score

## **LOAD AND TRANSFORM DATASET**

In [ ]:
df = load_dataset("keelezibel/hate-speech-indo")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/228 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/4.41M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15127 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3782 [00:00<?, ? examples/s]

In [ ]:
df

DatasetDict({
    train: Dataset({
        features: ['text', 'hs', 'abusive', 'hs_individual', 'hs_group', 'hs_religion', 'hs_race', 'hs_physical', 'hs_gender', 'hs_other', 'hs_weak', 'hs_moderate', 'hs_strong'],
        num_rows: 15127
    })
    test: Dataset({
        features: ['text', 'hs', 'abusive', 'hs_individual', 'hs_group', 'hs_religion', 'hs_race', 'hs_physical', 'hs_gender', 'hs_other', 'hs_weak', 'hs_moderate', 'hs_strong'],
        num_rows: 3782
    })
})

In [ ]:
df_train = df['train'].to_pandas()
df_test = df['test'].to_pandas()

In [ ]:
df = pd.concat([df_train, df_test], ignore_index=True)
df.head()

,text,hs,abusive,hs_individual,hs_group,hs_religion,hs_race,hs_physical,hs_gender,hs_other,hs_weak,hs_moderate,hs_strong
0,USER USER USER USER USER USER USER USER USER U...,0,1,0,0,0,0,0,0,0,0,0,0
1,RT USER Cebong akan terus membela junjungannya...,1,1,0,1,1,0,0,0,0,0,1,0
2,USER Terus bagaimana dgn uu MD3 bang?',0,0,0,0,0,0,0,0,0,0,0,0
3,"""\xf0\x9f\x91\xa6: """"Jangan jahatin anak orang...",1,1,1,0,0,0,1,0,0,1,0,0
4,Derita nun jauh disana akan sama dirasakan per...,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18909 entries, 0 to 18908
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   text           18909 non-null  object
 1   hs             18909 non-null  int64 
 2   abusive        18909 non-null  int64 
 3   hs_individual  18909 non-null  int64 
 4   hs_group       18909 non-null  int64 
 5   hs_religion    18909 non-null  int64 
 6   hs_race        18909 non-null  int64 
 7   hs_physical    18909 non-null  int64 
 8   hs_gender      18909 non-null  int64 
 9   hs_other       18909 non-null  int64 
 10  hs_weak        18909 non-null  int64 
 11  hs_moderate    18909 non-null  int64 
 12  hs_strong      18909 non-null  int64 
dtypes: int64(12), object(1)
memory usage: 1.9+ MB


## **CLEAN TEXT**

In [ ]:
def remove_duplicate_user_rt(text):
    words = text.split()
    unique_words = []

    for word in words:
        if word in {"USER", "RT"} and (not unique_words or unique_words[-1] != word):
            unique_words.append(word)
        elif word not in {"USER", "RT"}:
            unique_words.append(word)

    return " ".join(unique_words)

In [ ]:
df["text"] = df["text"].apply(remove_duplicate_user_rt)

In [ ]:
def clean_text(text):
    text = re.sub(r"\b(USER|RT|www|https)\b", "", text, flags=re.IGNORECASE)
    text = re.sub(r"\\x[a-fA-F0-9]{2}", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [ ]:
df["text"] = df["text"].apply(clean_text)

In [ ]:
df.head()

,text,hs,abusive,hs_individual,hs_group,hs_religion,hs_race,hs_physical,hs_gender,hs_other,hs_weak,hs_moderate,hs_strong
0,Keracunan tiap nete',0,1,0,0,0,0,0,0,0,0,0,0
1,Cebong akan terus membela junjungannya walaupu...,1,1,0,1,1,0,0,0,0,0,1,0
2,Terus bagaimana dgn uu MD3 bang?',0,0,0,0,0,0,0,0,0,0,0,0
3,""": """"Jangan jahatin anak orang""""\n: """"Baru jug...",1,1,1,0,0,0,1,0,0,1,0,0
4,Derita nun jauh disana akan sama dirasakan per...,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
def letters_only(text):
  text = re.sub(r"[^a-zA-Z]+", " ", text).strip().lower()
  return text

In [ ]:
df["text"] = df["text"].apply(letters_only)

In [ ]:
df.head()

,text,hs,abusive,hs_individual,hs_group,hs_religion,hs_race,hs_physical,hs_gender,hs_other,hs_weak,hs_moderate,hs_strong
0,keracunan tiap nete,0,1,0,0,0,0,0,0,0,0,0,0
1,cebong akan terus membela junjungannya walaupu...,1,1,0,1,1,0,0,0,0,0,1,0
2,terus bagaimana dgn uu md bang,0,0,0,0,0,0,0,0,0,0,0,0
3,jangan jahatin anak orang n baru juga sekali u...,1,1,1,0,0,0,1,0,0,1,0,0
4,derita nun jauh disana akan sama dirasakan per...,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
!wget -O kamusalay.csv https://raw.githubusercontent.com/okkyibrohim/id-multi-label-hate-speech-and-abusive-language-detection/master/new_kamusalay.csv

--2024-12-17 03:37:23--  https://raw.githubusercontent.com/okkyibrohim/id-multi-label-hate-speech-and-abusive-language-detection/master/new_kamusalay.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 285941 (279K) [application/octet-stream]
Saving to: ‘kamusalay.csv’

kamusalay.csv       100%[===================>] 279.24K  --.-KB/s    in 0.03s   

2024-12-17 03:37:24 (9.80 MB/s) - ‘kamusalay.csv’ saved [285941/285941]



In [ ]:
 df_slang = pd.read_csv('kamusalay.csv',header=None, encoding="latin1")

In [ ]:
df_slang.head()

,0,1
0,anakjakartaasikasik,anak jakarta asyik asyik
1,pakcikdahtua,pak cik sudah tua
2,pakcikmudalagi,pak cik muda lagi
3,t3tapjokowi,tetap jokowi
4,3x,tiga kali


In [ ]:
df_slang = df_slang.rename(columns={0: "nonformal", 1: "formal"})

In [ ]:
slang_dict = {df_slang.nonformal[i]: df_slang.formal[i] for i in range(0, len(df_slang.nonformal.values))}

In [ ]:
def replace_slang(sentence):
    tokens = sentence.split()
    for i, token in enumerate(tokens):
        if token in slang_dict.keys():
            tokens[i] = slang_dict[token]
    return ' '.join(tokens)

df['text'] = df['text'].apply(replace_slang)
df.head()

,text,hs,abusive,hs_individual,hs_group,hs_religion,hs_race,hs_physical,hs_gender,hs_other,hs_weak,hs_moderate,hs_strong
0,keracunan tiap nete,0,1,0,0,0,0,0,0,0,0,0,0
1,cebong akan terus membela junjungannya walaupu...,1,1,0,1,1,0,0,0,0,0,1,0
2,terus bagaimana dengan undang undang md bang,0,0,0,0,0,0,0,0,0,0,0,0
3,jangan menjahati anak orang dan baru juga seka...,1,1,1,0,0,0,1,0,0,1,0,0
4,derita nun jauh di sana akan sama dirasakan pe...,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_multi = df[['text', 'abusive', 'hs_individual', 'hs_group', 'hs_religion', 'hs_race', 'hs_other']]

In [ ]:
df_strengthness = df[['text', 'hs_weak', 'hs_moderate', 'hs_strong']]
df_strengthness.replace(["", "None", "null"], np.nan, inplace=True)

In [ ]:
df_multi.replace(["", "None", "null"], np.nan, inplace=True)
df_strengthness.replace(["", "None", "null"], np.nan, inplace=True)

<ipython-input-23-97d0b573e28b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_multi.replace(["", "None", "null"], np.nan, inplace=True)


In [ ]:
df_multi.dropna(subset=['text'], inplace=True)
df_strengthness.dropna(inplace=True)

<ipython-input-25-9347b2fd06e0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_multi.dropna(subset=['text'], inplace=True)


## **Splitting Dataset For MultiLabel Hatespeech Classification**

In [ ]:
df_multi_train = df_multi.iloc[:15127]
df_multi_val_test = df_multi.iloc[15127:]

In [ ]:
df_multi_val, df_multi_test = train_test_split(df_multi_val_test, test_size=0.2, random_state=42)

In [ ]:
df_multi_train.to_csv('multilabel_train.csv', index=False)
df_multi_val.to_csv('multilabel_val.csv', index=False)
df_multi_test.to_csv('multilabel_test.csv', index=False)

## **Splitting Dataset for MultiClass Hatespeech Strength Classification**

In [ ]:
df_strong = df_strengthness[df_strengthness['hs_strong']==1]
df_weak = df_strengthness[df_strengthness['hs_weak']==1]
df_moderate = df_strengthness[df_strengthness['hs_moderate']==1]

In [ ]:
df_sample_strong = df_strong.sample(n=600, random_state=42)
df_sample_weak = df_weak.sample(n=600, random_state=42)
df_sample_moderate = df_moderate.sample(n=600, random_state=42)

In [ ]:
df_sample = pd.concat([df_sample_strong, df_sample_weak, df_sample_moderate], ignore_index=True)

In [ ]:
shuffled_indices = np.random.permutation(len(df_sample))

In [ ]:
shuffled_df = df_sample.iloc[shuffled_indices]

In [ ]:
df_strengthness_train, df_strengthness_test = train_test_split(shuffled_df, test_size=0.2, random_state=42)

In [ ]:
df_strengthness_train, df_strengthness_val = train_test_split(df_strengthness_train, test_size=0.2, random_state=42)